In [ ]:
import geopandas as gp

In [ ]:
cua_feather = gp.read_feather(r'../output/Census_Urban_Areas_index.feather')
crash_feather = gp.read_feather(r'../output/Crash_index.feather')
district_feather = gp.read_feather(r'../output/VDOT_Districts_index.feather')

In [ ]:
district_feather.head(2)

In [ ]:
crash_feather.head()

In [ ]:
crash_feather.dtypes

In [ ]:
# Count crash by district and year
district_crash_year = gp.sjoin(district_feather, crash_feather).groupby(['district_code', 'crash_year']).size().reset_index()
district_crash_year.head()

In [ ]:
district_crash_year = district_crash_year.rename(columns={0: 'crash_count'})

In [ ]:
district_crash_year

In [ ]:
# Count crash by district
intersection_cnt = gp.sjoin(district_feather, crash_feather).groupby('district_code').size().reset_index()
intersection_cnt.head()

In [ ]:
# Merge with district and plot
intersection_cnt = intersection_cnt.rename(columns={0: 'crash_count'})
district_feather = district_feather.merge(intersection_cnt, on='district_code')
district_feather

In [ ]:
district_feather.plot(column='crash_count', cmap='Blues', linewidth=0.8, edgecolor='0.8')

In [ ]:
cua_feather.head()

In [ ]:
# Count crash by cencus bloack area
cua_crash_cnt = gp.sjoin(cua_feather, crash_feather).groupby('uace10').size().reset_index()
cua_crash_cnt = cua_crash_cnt.rename(columns={0: 'crash_count'})
cua_feather = cua_feather.merge(cua_crash_cnt, on='uace10')
cua_feather.plot(column='crash_count', cmap='Blues', linewidth=0.8, edgecolor='0.8')

In [ ]:
mpo = gp.read_feather(r'../output/MPO_Study_Areas_index.feather')
mpo.head()

In [ ]:
mpo

In [ ]:
# Count crash by mpo
mpo_crash_cnt = gp.sjoin(mpo, crash_feather).groupby('mpo_id').size().reset_index()
mpo_crash_cnt = mpo_crash_cnt.rename(columns={0: 'crash_count'})
mpo = mpo.merge(mpo_crash_cnt, on='mpo_id')
mpo.plot(column='crash_count', cmap='Blues', linewidth=0.8, edgecolor='0.8')

In [ ]:
import plotly.express as px

In [ ]:
import json
json.loads(district_feather.to_json()).keys()

In [ ]:
fig = px.choropleth_mapbox(geojson=json.loads(district_feather.to_json()), locations=district_feather.district_code, color=district_feather.crash_count,
hover_name=district_feather.district_name,
                           #color_continuous_scale="Viridis",
                           range_color=(0, district_feather['crash_count'].max()),
                           mapbox_style="carto-positron",
                           zoom=5, center = {"lat": 37.4316, "lon": -78.6569},
                           opacity=0.5,
                           color_continuous_scale='blues',
                           labels={'crash_count':'crash count'},
                           title='District Crash Count'
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

district count animation by year

In [ ]:
district_json = json.loads(district_feather.to_json())

In [ ]:
fig = px.choropleth_mapbox(district_crash_year, geojson=district_json, locations='district_code', color='crash_count',
                           #color_continuous_scale="Viridis",
                           range_color=(0, district_crash_year['crash_count'].max()),
                           featureidkey="properties.district_code",
                           mapbox_style="carto-positron",
                           zoom=5, center = {"lat": 37.4316, "lon": -78.6569},
                           opacity=0.5,
                           color_continuous_scale='blues',
                           labels={'crash_count':'crash count'},
                           title='District Crash Count',
                           animation_frame='crash_year'
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()